In [7]:
from bs4 import BeautifulSoup as soup
from lxml import etree as ET
import pandas as pd
import requests
from tqdm.notebook import tqdm
from time import sleep
import time

In [36]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content)
    records = xml.find_all('record', {'type':'Bibliographic'})
    results = xml.find_all("numberofrecords")[0].text
    results = int(results)
    
    if results < 100:
        
        return records
    
    else:
        num_results = 100
        i = 101
        runs = results//100
        count=0
        pbar = tqdm(total=runs)
        
        while num_results == 100:        
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content)
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            count+=1
            num_results = len(new_records)
            pbar.update()
            
            if count%50 == 0:
                sleep(10)
            else:
                continue
                        
        return records
        

In [38]:
myquery0 = dnb_sru('HSS=diss* and (hsg="3*" or hsg="5*" or hsg="6*") and jhr="2000"')
#myquery1 =  dnb_sru('HSS=diss* and (hsg="5*") and jhr="1990"')
print(len(myquery0), "Ergebnisse")

201


  0%|          | 0/201 [00:00<?, ?it/s]

20152 Ergebnisse


In [39]:
print(myquery1[1])

<record type="Bibliographic" xmlns="http://www.loc.gov/MARC21/slim">
<leader>00000nam a2200000 c 4500</leader>
<controlfield tag="001">901498319</controlfield>
<controlfield tag="003">DE-101</controlfield>
<controlfield tag="005">20171202153622.0</controlfield>
<controlfield tag="007">tu</controlfield>
<controlfield tag="008">901127s1990    gw ||||| |||| 00||||eng  </controlfield>
<datafield ind1=" " ind2=" " tag="015">
<subfield code="a">90,A50,0848</subfield>
<subfield code="2">dnb</subfield>
</datafield>
<datafield ind1="7" ind2=" " tag="016">
<subfield code="2">DE-101</subfield>
<subfield code="a">901498319</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="020">
<subfield code="a">9783880992399</subfield>
<subfield code="c">kart. : DM 40.00</subfield>
<subfield code="9">978-3-88099-239-9</subfield>
</datafield>
<datafield ind1=" " ind2=" " tag="020">
<subfield code="a">3880992398</subfield>
<subfield code="c">kart. : DM 40.00</subfield>
<subfield code="9">3-88099-239-8</sub